<a href="https://colab.research.google.com/github/meesumabidi/NAVTCC/blob/main/sirasim_whatsappchat_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install emoji


In [ ]:


import re
import pandas as pd
import numpy as np
import emoji
from collections import Counter
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk import *
from datetime import date
import plotly.express as px
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import text2emotion as te

# Extract Time
def date_time(s):
    pattern = '^([0-9]+)(\/)([0-9]+)(\/)([0-9]+), ([0-9]+):([0-9]+)[ ]?(AM|PM|am|pm)? -'
    result = re.match(pattern, s)
    if result:
        return True
    return False

# Find Authors or Contacts
def find_author(s):
    s = s.split(":")
    if len(s)==2:
        return True
    else:
        return False

# Finding Messages
def getDatapoint(line):
    splitline = line.split(' - ')
    dateTime = splitline[0]
    date, time = dateTime.split(", ")
    message = " ".join(splitline[1:])
    if find_author(message):
        splitmessage = message.split(": ")
        author = splitmessage[0]
        message = " ".join(splitmessage[1:])
    else:
        author= None
    return date, time, author, message


# In[4]:


data = []
conversation = 'whatsapp chat.txt'
with open(conversation, encoding="utf-8") as fp:
    fp.readline()
    messageBuffer = []
    date, time, author = None, None, None
    while True:
        line = fp.readline()
        if not line:
            break
        line = line.strip()
        if date_time(line):
            if len(messageBuffer) > 0:
                data.append([date, time, author, ' '.join(messageBuffer)])
            messageBuffer.clear()
            date, time, author, message = getDatapoint(line)
            messageBuffer.append(message)
        else:
            messageBuffer.append(line)


# # unsupervised learning 

# In[8]:


df = pd.DataFrame(data, columns=['Date', 'Time', 'Author', 'Message'])
df['Date'] = pd.to_datetime(df['Date'])

df
# data = df.dropna()
# # data
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
# sentiments = SentimentIntensityAnalyzer()
# data["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data["Message"]]
# data["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data["Message"]]
# data["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data["Message"]]
# print(data.head())


# # creating new columns year month day time hour minute weeday

# In[12]:


df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month_name()
df['day'] = df['Date'].dt.day
df['Time'] = pd.to_datetime(df['Time'])
df['hour'] = df['Time'].dt.hour
df['minute'] = df['Time'].dt.minute
df['weekday'] = df['Date'].dt.day_name()

df.head()


# # # split URL, letter and word count 

# In[13]:



#column urlcount
URLPATTERN = r'(https?://\S+)'
df['urlcount'] = df.Message.apply(lambda x: re.findall(URLPATTERN, x)).str.len()

#column Letter_Count
df['Letter_Count'] = df['Message'].apply(lambda s : len(s))
#column Word_Count
df['Word_Count'] = df['Message'].apply(lambda s : len(s.split(' ')))
df.head()


# # extract emoji in message

# In[14]:



def split_count(text):
    emoji_list = []
    data = re.findall(u'[\U0001f300-\U0001f650]|[\u2000-\u3000]', text)
#     data = re.findall(u'(?:\uD83C[\uDF00-\uDFFF])|(?:\uD83D[\uDC00-\uDDFF])', text)
    
#     print(data)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)
#             print(emoji_list)
    return emoji_list

df["emoji"] = df["Message"].apply(split_count)
df.head(50)


# # chart shows the number of messages on weekdays and weekends (Time Series)

# In[15]:


date_grouped = df.groupby('weekday')['Message'].count().plot(kind='line', figsize=(20,10), color='#A26360')


# # chart shows the number of messages date wise

# In[16]:


date_grouped = df.groupby('Date')['Message'].count().plot(kind='line', figsize=(20,10), color='#A26360')


# # favorite days to chat with friends

# In[17]:


weekday_grouped_msg =  (df.set_index('weekday')['Message']
                          .groupby(level=0)
                          .value_counts()
                          .groupby(level=0)
                          .sum()
                          .reset_index(name='count'))
weekday_grouped_msg

fig = px.line_polar(weekday_grouped_msg, r='count', theta='weekday', line_close=True)
fig.update_traces(fill='toself')
fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
    )),
  showlegend=False
)
fig.show()


# # Total Messages per hour

# In[18]:


hour_grouped_msg =  (df.set_index('hour')['Message']
                          .groupby(level=0)
                          .value_counts()
                          .groupby(level=0)
                          .sum()
                          .reset_index(name='count'))
fig = px.bar(hour_grouped_msg, x='hour', y='count',
                 Labels={'hour':'24 Hour Period'}, 
                 height=400)
fig.update_traces(marker_color='#EDCC8B', marker_line_color='#D4A29C',
                  marker_line_width=1.5, opacity=0.6)
fig.update_layout(title_text='Total Messages by Hour of the Day')
fig.show()


# # pivot table day wise and month wise

# In[21]:


grouped_by_month_and_day = df.groupby(['month', 'weekday'])['Message'].value_counts().reset_index(name='count')
grouped_by_month_and_day
months= ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
pt = grouped_by_month_and_day.pivot_table(index= 'month', columns= 'weekday', values='count').reindex(index=months, columns= days)
fig = px.imshow(pt, Labels=dict(x="Day of Week", y="Months", color="Count"), x=days y=months)
fig.update_layout(
    width = 700, height = 700)
fig.show()


# # which author sent maximum messages 

# In[25]:


qty_message_author = df['Author'].value_counts()
qty_message_author.plot(kind='barh',figsize=(20,10), color=['#D4A29C', '#E8B298', '#EDCC8B', '#BDD1C5', '#9DAAA2'])
qty_message_author.head(10)


# # find common words and their frequency

# In[29]:


commond_words = df[['Author','Message']].copy()

from nltk.corpus import stopwords
STOPWORDS = stopwords.words('english')

stopwords = list(STOPWORDS)
extra = ["<multimedia", "omitido>", "k", "d","si","multimedia", "omitido"]
stopwords = stopwords + extra
commond_words["Message"] = (commond_words["Message"]
                           .str.lower()
                           .str.split()
                           .apply(lambda x: [item for item in x if item not in stopwords])
                           .explode()
                           .reset_index(drop=True)
                 )

# commond_words['Message']= commond_words['Message'].apply(remove_emoji)
commond_words['Message']= commond_words['Message'].replace('nan', np.NaN)
commond_words['Message']= commond_words['Message'].replace('', np.NaN)
commond_words['Message']= commond_words.Message.str.replace(r"(a|j)?(ja)+(a|j)?", "jaja",regex=True)
commond_words['Message']= commond_words.Message.str.replace(r"(a|j)?(jaja)+(a|j)?", "jaja",regex=True)


words_dict = dict(Counter(commond_words.Message))
words_dict = sorted(words_dict.items(), key=lambda x: x[1], reverse=True)

words_dict = pd.DataFrame(words_dict, columns=['words', 'count'])

fig = px.bar(words_dict.head(10).dropna(), x='words', y='count',
                 Labels={'words':'Common Words'}, 
                 height=400)
fig.update_traces(marker_color='#EDCC8B', marker_line_color='#D4A29C',
                  marker_line_width=1.5, opacity=0.6)
fig.update_layout(title_text='Commond Words Chart')
fig.show()


# In[35]:


#function to display wordcloud
def plot_cloud(wordcloud):
    # Set figure size
    plt.figure(figsize=(40, 30))
    # Display image
    plt.imshow(wordcloud) 
    # No axis details
    plt.axis("off");

#function to remove urls from text
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)


# # word cloud

# In[33]:


chat_word_cloud = df[['Message']].copy()
# chat_word_cloud['Message']= chat_word_cloud['Message'].apply(remove_emoji)
chat_word_cloud['Message']= chat_word_cloud['Message'].apply(remove_urls)
chat_word_cloud['Message']= chat_word_cloud['Message'].replace('nan', np.NaN)
chat_word_cloud['Message']= chat_word_cloud['Message'].replace('', np.NaN)
chat_word_cloud['Message']= chat_word_cloud.Message.str.replace(r"(a|j)?(ja)+(a|j)?", "jaja",regex=True)
chat_word_cloud['Message']= chat_word_cloud.Message.str.replace(r"(a|j)?(jaja)+(a|j)?", "jaja",regex=True)
text = " ".join(review for review in chat_word_cloud.Message.dropna())
wordcloud = WordCloud(width = 3000, height = 2000, random_state=1, 
background_color='black', colormap='Set2', collocations=False,
stopwords = stopwords).generate(text)
# Plot
plot_cloud(wordcloud)


# In[6]:


df.head()


# In[38]:


# df = pd.DataFrame(data, columns=["Date", 'Time', 'Author', 'Message'])
# df['Date'] = pd.to_datetime(df['Date'])
# import nltk
# nltk.download('vader_lexicon')
data = df.dropna()

sentiments = SentimentIntensityAnalyzer()
data["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data["Message"]]
data["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data["Message"]]
data["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data["Message"]]

print(data.head(50))


# In[43]:


for i in data["Message"]:
    te.get_emotion(i)
text = "I was asked to sign a third party contract a week out from stay. If it wasn't an 8 person group that took a lot of wrangling I would have cancelled the booking straight away. Bathrooms - there are no stand alone bathrooms. Please consider this - you have to clear out the main bedroom to use that bathroom. Other option is you walk through a different bedroom to get to its en-suite. Signs all over the apartment - there are signs everywhere - some helpful - some telling you rules. Perhaps some people like this but It negatively affected our enjoyment of the accommodation. Stairs - lots of them - some had slightly bending wood which caused a minor injury."
te.get_emotion(text)
# sentiments.polarity_scores(i)["pos"] for i in data["Message"]


# In[ ]:






SyntaxError: ignored